In [7]:
### Use "git rebase -i master" when/if the previous branch gets merged

import numpy as np
import pandas as pd
import sys
import math
import os

sys.path.insert(0, "/Users/bryceturner/Personal_Projects/MagicNFL/src/main")
from field_square import *
from football_field import *
from football_field_utils import *



os.chdir("../")
os.getcwd()

'/Users/bryceturner/Personal_Projects/MagicNFL'

In [8]:
os.getcwd()

'/Users/bryceturner/Personal_Projects/MagicNFL'

In [ ]:
# Things we'll need
# way to convert from "yardline and team-side" to x,y coordinates
#  -> OR is it better to define the ball usign the Center? 

In [9]:
using_data = (
    pd.read_csv("input/week1.csv")
    .query("gameId==2021090900")
)

using_data.head()

players_data = (
    pd.read_csv("input/players.csv")
)



In [10]:
using_data.head(4)

,gameId,playId,nflId,frameId,time,jerseyNumber,team,playDirection,x,y,s,a,dis,o,dir,event
0,2021090900,97,25511.0,1,2021-09-10T00:26:31.100,12.0,TB,right,37.77,24.22,0.29,0.30,0.03,165.16,84.99,None
1,2021090900,97,25511.0,2,2021-09-10T00:26:31.200,12.0,TB,right,37.78,24.22,0.23,0.11,0.02,164.33,92.87,None
2,2021090900,97,25511.0,3,2021-09-10T00:26:31.300,12.0,TB,right,37.78,24.24,0.16,0.10,0.01,160.24,68.55,None
3,2021090900,97,25511.0,4,2021-09-10T00:26:31.400,12.0,TB,right,37.73,24.25,0.15,0.24,0.06,152.13,296.85,None


In [12]:
## Arbitrary Inputs. Will need to be validated later
side_length=1
pocket_depth=10 #x_value, measured in yards --- currently, this goes 10 yards IN BOTH DIRECTIONS(i.e. down field)
pocket_width=10 #y_value, measured in yards

football_starting_square = \
    (using_data[(using_data.team=="football") 
                & (using_data.frameId==1)]
    .apply(lambda center: find_center_of_square_containing((center["x"], center["y"]), side_length), axis=1)
    )

xlims = [(center[0]-pocket_depth, center[0]+pocket_depth) for center in football_starting_square]
ylims = [(center[1]-pocket_depth, center[1]+pocket_depth) for center in football_starting_square]

play_pockets=[]
for x, y in zip(xlims, ylims):
    tmp = football_field(x, y, side_length)
    tmp.set_field_squares()
    play_pockets.append(tmp)


play_pockets[0].field_squares[0]

In [27]:
class play_pocket(football_field):

    def __init__(self, football_starting_coordinates, side_length, offense_direction, gameId, playId):
        
        if type(football_starting_coordinates) is not tuple:
            raise TypeError("football_starting_coordinates must be a tuple of the form (x,y)")
            
        if offense_direction not in ["left", "right"]:
            raise ValueError("offensive direction must be either 'left' or 'right'")

        self.football_starting_coordinates=football_starting_coordinates
        self.offense_direction=offense_direction.lower()
        self.gameId=gameId
        self.playId=playId
        self.side_length=side_length
        self.pocket_square_center=find_center_of_square_containing(self.football_starting_coordinates, side_length)

    def set_pocket_limits(self, pocket_depth, pocket_width):
        """ Pocket_depth is the x value, because it goes towards the endzone (i.e. the x-value)
        """
        # X-Values -- Depends on which direction the offense is going
        if self.offense_direction=="left":
            self.xlims=(self.pocket_square_center[0], self.pocket_square_center[0]+pocket_depth)
        elif self.offense_direction=="right":
            self.xlims=(self.pocket_square_center[0]-pocket_depth, self.pocket_square_center[0])
        else:
            raise ValueError("Offense Direction must be either 'left' or 'right'")
        # Y-Values are independent of the play direction
        self.ylims=(self.pocket_square_center[1]-pocket_width/2, self.pocket_square_center[1]+pocket_width/2)


        
    

    

        

In [28]:
tmp = play_pocket((4, 25), 1,  "left", 1, 97)
tmp.set_pocket_limits(10, 7)
tmp.set_field_squares()

tmp.field_squares[0].center

(5.0, 22.5)

In [175]:
if "br" not in ["left", "right"]:
    raise ValueError("offensive direction must be either 'left' or 'right'")

ValueError: offensive direction must be either 'left' or 'right'